In [1]:
import os

# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
spark_version = 'spark-3.4.0'
os.environ['SPARK_VERSION']=spark_version

# Set Environment Variables
os.environ["JAVA_HOME"] = "/opt/homebrew/opt/openjdk/bin/java"
os.environ["SPARK_HOME"] = "/opt/homebrew/Caskroom/miniforge/base/bin/pyspark"

# Start a SparkSession
import findspark
findspark.init()

Exception: Unable to find py4j in /opt/homebrew/Caskroom/miniforge/base/bin/pyspark/python, your SPARK_HOME may not be configured correctly

In [12]:
!which pyspark

/opt/homebrew/Caskroom/miniforge/base/bin/pyspark


In [ ]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2023-01-26 02:38:44--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.2’

postgresql-42.2.16. 100%[===================>] 979.38K  6.15MB/s    in 0.2s    

2023-01-26 02:38:44 (6.15 MB/s) - ‘postgresql-42.2.16.jar.2’ saved [1002883/1002883]



In [15]:
import pyspark

In [17]:
spark = pyspark.sql.SparkSession.builder.appName("AmazonReviewsETL").getOrCreate()

AttributeError: module 'pyspark' has no attribute 'sql'

In [16]:
# start the spark session
spark = pyspark.SparkSession.builder.appName("Amazon-Vine-Analysis").getOrCreate()

AttributeError: module 'pyspark' has no attribute 'SparkSession'

### Load Amazon Data into Spark DataFrame

In [18]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Home_Entertainment_v1_00.tsv.gz"
spark.sparkContext.addFile(url)

# Read in the Review dataset as a DataFrame
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

ImportError: cannot import name 'SparkFiles' from 'pyspark' (unknown location)

### Create DataFrames to match tables

In [ ]:
# Create the customers_table DataFrame
from pyspark.sql.functions import count
customers_df = df.groupby("customer_id").agg(count("customer_id")).withColumnRenamed("count(customer_id)", "customer_count")
customers_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   10142992|             1|
|   16457323|             6|
|   11935383|             1|
|   46277736|             1|
|   13671072|             1|
|   21453814|             1|
|   17684885|             1|
|   20415768|             1|
|   15212710|             1|
|    5220924|             1|
|   46253451|             6|
|     971908|             1|
|   32829933|             1|
|   51221518|             1|
|   12002637|             2|
|   16105308|             1|
|     135867|             1|
|   47425808|             1|
|   43138273|             1|
|   16411995|             1|
+-----------+--------------+
only showing top 20 rows



In [ ]:
# Create the products_table DataFrame and drop duplicates. 

products_df = df.select(['product_id','product_title']).drop_duplicates()
products_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00N9OT6RM|Upstar 19-Inch 72...|
|B007R9RUPU|Kinivo LS210 Port...|
|B00U9U9AAM|Samsung J6200, SB...|
|B00QCLTOQM|Panasonic TC32A40...|
|B007F9XJW0|Sony DVPFX780 7-I...|
|B00JA7ZQOY|Minix X7mini Andr...|
|B008I641TE|SquareTrade 2-Yea...|
|B00EJ5UJZ8|PowerSmart 4200mA...|
|B006L8TX94|TiVo Premiere 500...|
|B00R8K9ZH4|Fosmon HYBO-DUOC ...|
|B001JHJK22|Mediabridge - RCA...|
|B00CWEJ5BW|2-Year Electronic...|
|B000BSHLLW|Ziotek Purse Lock...|
|B0019OJOTE|FAVI 32-Inch 1080...|
|B00MOCT3NW|BenQ MH630 1.4A 1...|
|B00VWV1O3S|12V 8Ah SLA Batte...|
|B00BBAG0DY|LG Electronics LA...|
|B00BXF7I8I|Seiki 1080p 60Hz ...|
|B0011ZOZ36|Vizio VW26LHDTV20...|
|B00Q8DB4YY|Atoah MXIII TV Bo...|
+----------+--------------------+
only showing top 20 rows



In [ ]:
# Create the review_id_table DataFrame. 
# Convert the 'review_date' column to a date datatype with to_date("review_date", 'yyyy-MM-dd').alias("review_date")
from pyspark.sql.functions import to_date
review_id_df = df.select(
    ['review_id',
     "customer_id",
     "product_id",
     "product_parent",
     to_date("review_date", 'yyyy-MM-dd').alias("review_date")
     ])

review_id_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
| RY01SAV7HZ8QO|     179886|B00NTI0CQ2|     667358431| 2015-08-31|
|R1XX8SDGJ4MZ4L|   37293769|B00BUCLVZU|     621695622| 2015-08-31|
|R149Q3B5L33NN5|    8332121|B00RBX9D5W|     143071132| 2015-08-31|
|R2ZVD69Z6KPJ4O|   47054962|B00UJ3IULO|     567816707| 2015-08-31|
|R1DIKG2G33ZLNP|   23413911|B0037UCTXG|     909557698| 2015-08-31|
|R3L6FGKAW0EYFI|    4417771|B004N866SU|     414565179| 2015-08-31|
| RAO0QZH5VC6VI|   47900707|B00JE6AOJS|     798450889| 2015-08-31|
|R25IK0UAHWNB22|   34112894|B00COL0B7A|     777554234| 2015-08-31|
|R2A9IHKZMTMAL1|   20691979|B00QHLSKOE|     885228855| 2015-08-31|
| R5XVKTHL6SITI|   25983343|B00UNL2MUW|     164482798| 2015-08-31|
|R2QZZOSTDDY1IE|   35816068|B00RIC9JB4|     184834831| 2015-08-31|
|R38CUDCFPSNYTD|   10628020|B00HPMCO6O|     444378461| 2015-08

In [ ]:
# Create the vine_table. DataFrame
vine_df = df.select([
    'review_id',
    'star_rating',
    'helpful_votes',
    'total_votes',
    'vine',
    'verified_purchase'
    ])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RY01SAV7HZ8QO|          4|            0|          0|   N|                Y|
|R1XX8SDGJ4MZ4L|          5|            0|          0|   N|                N|
|R149Q3B5L33NN5|          5|            0|          0|   N|                Y|
|R2ZVD69Z6KPJ4O|          1|            0|          2|   N|                Y|
|R1DIKG2G33ZLNP|          4|            0|          0|   N|                Y|
|R3L6FGKAW0EYFI|          1|            1|          1|   N|                N|
| RAO0QZH5VC6VI|          1|            0|          0|   N|                Y|
|R25IK0UAHWNB22|          3|            0|          0|   N|                Y|
|R2A9IHKZMTMAL1|          5|            1|          2|   N|                Y|
| R5XVKTHL6SITI|          5|            0|          0|   N|     

### Connect to the AWS RDS instance and write each DataFrame to its table. 

In [ ]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://dataviz.cozcjayx8uch.us-west-2.rds.amazonaws.com:5432/Challenge_Analysis"
config = {"user":"user_name", 
          "password": "******", 
          "driver":"org.postgresql.Driver"}

In [ ]:
# Write review_id_df to table in RDS
review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [ ]:
# Write products_df to table in RDS
products_df.write.jdbc(url=jdbc_url, table='products_table', mode=mode, properties=config)

In [ ]:
# Write customers_df to table in RDS
customers_df.write.jdbc(url=jdbc_url, table='customers_table', mode=mode, properties=config)

In [ ]:
# Write vine_df to table in RDS
vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)

In [ ]:
# End the session
spark.stop()